In [ ]:
!pip install ipywidgets

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset
notebook_login()

In [ ]:
import json
import os
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

# Create the "output" folder if it doesn't exist
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

dataset_list = [
    {"hf_datasets": "wikimedia/wikipedia", "hf_corpus_paths": "20231101.kn"},
    {"hf_datasets": "mc4", "hf_corpus_paths": "kn"},
    {"hf_datasets": "uonlp/CulturaX", "hf_corpus_paths": "kn"},
]
all_data_points = []  # List to store all data points

for dataset_config in dataset_list:
    hf_datasets = dataset_config["hf_datasets"]
    hf_corpus_paths = dataset_config["hf_corpus_paths"]

    dataset_identifier = hf_datasets.split("/")[-1]  # Consider only the right half after '/'

    dataset = load_dataset(hf_datasets, hf_corpus_paths, split="train", trust_remote_code=True)

    current_data_points = []  # List to store data points from the current dataset

    for data_point in dataset:
        text = data_point["text"]
        source = dataset_identifier  # Use the modified dataset identifier

        json_entry = {
            "text": text,
            "source": source
        }

        current_data_points.append(json_entry)  # Add the data point to the current dataset list
        all_data_points.append(json_entry)  # Add the data point to the overall list

    # Save the data points from the current dataset to a JSON file in the "output" folder
    output_file_path = os.path.join(output_folder, f"{dataset_identifier}_output.json")
    with open(output_file_path, "w", encoding="utf-8") as json_file:
        json.dump(current_data_points, json_file, ensure_ascii=False, indent=2)

    print(f"Data points from {dataset_identifier} saved to {output_file_path}")

# Save the overall list of data points to a final JSON file in the "output" folder
final_output_file_path = os.path.join(output_folder, "all_data_points_output.json")
with open(final_output_file_path, "w", encoding="utf-8") as json_file:
    json.dump(all_data_points, json_file, ensure_ascii=False, indent=2)

print(f"All data points saved to {final_output_file_path}")


In [ ]:
dataset = load_dataset('json', data_files='./output/wikipedia_output.json' , split='train')

In [ ]:
print(dataset['text'][:10])

In [ ]:
dataset.push_to_hub("CognitiveLab/Project_K_TrainDataset_Small" ,private=True)